# Metaprogramming

Metaprogramming writes code that can modify itself. One natural application is for writing repetitive code: if you can come up, for example, for a rule of how to define a large number of arrays, perhaps you can write a code that writes the code to define all these arrays.

In the following note I will present: 
 - Expressions
 - Functions vs Macros 
 - Generated Functions

**Let's Start with the basics: Expressions**
***
Everything in Julia is an expression that returns a value when executed. In fact, every piece of program code is internally represented as an ordinary Julia data structure, referred to as an expression.

In [1]:
#Ex:
2+3 #is an expression

5

To make Julia see 2+3 as an expression and block its evaluation immediately (i.e., keep the expression from being executed to render a result), it must be quoted. Quoting happens in two ways.
 - using : operator
 - using quote-end operator
 
 
Alternatively, one could use the Expresison construcvtor EXPR

In [2]:
a = :(2+3) #this is an unevaluated expression (if we remove : we evalueate)
typeof(:(2+3))

#if the expression takes more than one line, we use the quote operator
a = quote
    a = 42 
    b = a^2
    a-b
end

#alternative one line version
:(a=42; b=a^2; a - b)


e1 = Expr(:call, *, 3, 4)

typeof(a)

Expr

**Note** Expr objects contain two parts:
 - A symbol identifying the kind of expression --> a.head
 - The expression arguments --> a.args

In [3]:
#we can look at the details of the expression with dump()
a = :(2+3)
dump(a)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 2
    3: Int64 3


In [4]:
#each of the fiels can be accessed with args()
a.args, a.head

(Any[:+, 2, 3], :call)

In [5]:
#each field can be modified
a.args[2] = 5
a.args

3-element Vector{Any}:
  :+
 5
 3

**Why do we need expressions?**
The key point of expression is that:
 - At the time an expression is constructed, not all the symbols have to be defined
 - However, they have to be at the time of evaluation (i.e., when eval() is invoked), otherwise errors occur.

In [7]:
e2 = Expr(:call, *, 3, :a)
try
	eval(e3) #evaluation of e3 has error because a is not assigned
catch e
    println(e)
end

UndefVarError(:e3)


In [8]:
# to fix this, we need to define a before evaluating the expression

e2 = Expr(:call, *, 3, :a)
a = 4
eval(e2)

12

**Note**: Expressions Can Change State of the Execution Environment.

Ex: expression e3 assigns a value to variable b when the expression e3 gets evaluated. 
It might even define b, if it does not already exist.

In [9]:
e3 = :(b = 1)

:(b = 1)

**Interpolating Expressions**

In [10]:
# To interpolate, we use the $ operator in an expression. 
a = 1
b = 99
e4 = :(a + b) #a will not be evaluated immediately > returns :(a+b)
e5 = :($a + b) #a will be evaluated immediately > returns :(1 + b)

:(1 + b)

**Thus**
 - Expression interpolation (with $) evaluates when the expression is constructed (at parse time)
 - Quotation (with : or quote-end) evaluates only when the expression is passed to eval at runtime.

## Functions Vs Macros

Functions and Macros are both programming structures in Julia. However:
 - Function: takes the input values and returns the computed values at runtime.
 - Macro: takes the input expression and returns the modified expressions at parse time. Macros are a form of custom program transformation

In [11]:
# A classical example

macro welcome(name)
    return :(println("Hello ", $name, " likes economics")) 
end
    
@welcome("Jesus")

Hello Jesus likes economics


In [12]:
#Evaluation at parse time

macro macex1(ex)
    quote
        println("First")
        println($ex)
        println("Third")
    end
end

@macex1 ("Second")

First
Second
Third


In [13]:
#Evaluation at run-time
macro macex1(ex)
    quote
        println("First")
        println(ex)
        println("Third")
    end
end

ex = "Second"

@macex1 ()

First
Second
Third


In [14]:
#The assert macro to the left takes an expression, ex, and tests whether it is true or not.

macro assert(ex)
    :($ex ? nothing : error("Assert failed: ", $(string(ex))))
end


try
	@assert 1 == 42
catch e
  println(e)
end

ErrorException("Assert failed: 1 == 42")


**Note** When you are using macros, it is important to keep an eye out for scoping issues.

In [15]:
macro f(x)
    quote
        s = 4
        (s, $(esc(s)))
    end
end

# This macro declares a variable s, and returns a quoted expression containing s and an escaped version of s.
# the escaped vesion of s is basically an s defined outside the macro function f(x)

s = 0

@f 5 #can use any number as x

(4, 0)

**Conventional use of macros**
 - Declare variables used in the macro as local - prevents conflict with other variables
 - Use the escape function esc to make sure that an interpolated expression is not expanded, but instead is used literally
 - Don't call eval inside a macro (because it is likely that the variables you are evaluating don't even exist at that point)

In [16]:
# A Good Example of a Hygienic Macro is the timeit Macro

using Printf
macro timeit(ex)
    quote
        local t0 = time()
        local val = $(esc(ex)) #$esc(ex)) expression allows you to 'escape' the code you want to time, 
        local t1 = time()
        println("elapsed time in seconds: ")
        @printf "%.3f" t1-t0
        val
    end
end

#= 
 the $esc(ex)) expression allows you to 'escape' the code you want to time, which is in ex, 
    so that it isn't evaluated in the macro, but left intact until the entire quoted expression
    is returned by @timeit macro is evaluated at the calling context and executed there.
=#

@timeit factorial(10)

elapsed time in seconds: 
0.016

3628800

### Generated Functions
***
Julia's generated functions provide a flexible way to move work from run time to compile time. As with macros, generated functions improve performance by moving constant parts of the computation earlier in the compilation stage. However, the big difference between generated functions and macros is that computations are fixed only for a certain type of argument for generated functions.
 - The function declaration MUST be annotated with the @generated macro.
 - In the body of the generated function you only have access to the types of the arguments – not their values --and any function that was defined before the definition of the generated function.
 - Generated functions only return a quoted expression, which when evaluated, performs the calculation and returns a result as desired with the given arguments.
 - Generated functions MUST NOT mutate or observe any non-constant global state, meaning they can only read global constants and cannot have any side effects. Generated function have to be completely pure. PURE in the sense that as of the time of this writing, they cannot define a closure or generator.

In [17]:
@generated function simplegen(x)
    Core.println(x)
    return :(x * x) #the function returns a quoted expression
end


#Output is from the println() statement in the return of the expression being evaluated
simplegen(4) #only prints 4
x = simplegen(2) 

Int64


4

In [18]:
y = simplegen("Aloha")

String


"AlohaAloha"

In [19]:
# Example 3: Generated functions and Method Redefinitions
origfunc(x) = "original definition"

#Let's define other operations using orgfunc
copyorigfunc(x) = origfunc(x) #Normal Function

@generated gen1(x) = origfunc(x) #generated function
@generated gen2(x) = :(origfunc(x)) #generated function

#let's add a new mothod for origfunc()
origfunc(x::Int) = "definition for Int"
origfunc(x::Type{Int}) = "definition for Type{Int}"

origfunc (generic function with 3 methods)

In [20]:
# origfunc(1) - here we see the new definition using an Integer 
#               argument takes precedence.
origfunc(1)

"definition for Int"

In [21]:
# copyorigfunc(1)  - follows from the assignment of copyorigfunc = origfunc 
#                    they point to the same place in memory.
copyorigfunc(1)

"definition for Int"

In [22]:
# gen1(1) - follows the origfunc(x) prior to the new 
#          definition of origfunc(x) 
gen1(1)

"original definition"

In [23]:
# gen2(1) - uses a quoted version of origfunc(x) which 
#           follows the latest definition during call invocation.
gen2(1)

"definition for Int"

**Note**: Generated functions have there their own view of defined functions


In [24]:
@generated gen1(x::Real) = origfunc(x)
gen1(1)

"definition for Type{Int}"